This notebook takes in the raw probabilities file (from `content/posts/2023/2023-12-17-probability-estimates-2024`) and:
1. Transforms it so that each person gets a row, and each event is a column
2. Adds a row of "average" predictions at the bottom
3. Adds a set of *resolutions*, answers as to whether each predicted event actually occurred.
4. Calculates Brier scores for each person as well as for the average predictions.

In [1]:
import polars as pl

In [2]:
raw = pl.read_csv('raw_probabilities.csv')

In [3]:
raw

Timestamp,What's your name?,Can I track your answers on my website alongside others'?,Geopolitics | China lands ground forces on the island of Taiwan,Geopolitics | Russia and Ukraine sign a truce to stop all combat operations and honor it until the end of the year,Geopolitics | Nuclear weapons are used in an attack (not just tested) somewhere in the world.,Geopolitics | Xi Jinping is still the premier of China at the end of 2024.,Geopolitics | Narendra Modi is still the prime minister of India at the end of 2024.,Geopolitics | Benjamin Netanyahu is still the prime minister of Israel at the end of 2024.,Geopolitics | Vladimir Putin is still the president of Russia at the end of 2024.,Geopolitics | Vlodymyr Zelenskyy is still the president of Ukraine at the end of 2024.,Geopolitics | Pope Francis (Jorge Mario Bergoglio) is still the active pope at the end of 2024.,US Politics | Donald Trump is elected president.,US Politics | Joe Biden is re-elected president.,US Politics | A Republican candidate who isn't Donald Trump is elected president.,US Politics | A Democrat candidate who isn't Joe Biden is elected president.,US Politics | At least one third-party US presidential candidate gets 3% or more of the popular vote.,US Politics | The US president elect dies between the election and the end of 2024.,US Politics | 2024 elections give Democrats control of the Senate.,US Politics | 2024 elections give Democrats control of the House of Representatives.,US Politics | At least one member of the US Supreme Court steps down or dies during 2024.,US Politics | President Jimmy Carter is still alive at the end of 2024.,US Politics | Donald Trump is convicted of a federal crime.,US Politics | Joe Biden is impeached.,Sports & Culture | Mainstream news outlets report definitively that Taylor Swift and Travis Kelce have broken up.,Sports & Culture | Taylor Swift releases an album of composed primarily of entirely new music.,"Sports & Culture | LeBron James retires from or otherwise leaves the NBA, and does not change his mind before the end of the year.","Sports & Culture | LeBron James and his son LeBron Jr (aka """"Bronny"""") are under contract with the same NBA team at any point during 2024.","Sports & Culture | Lionel Messi retires from or otherwise leaves the MLS, and does not change his mind before the end of the year.",Sports & Culture | The US wins the most gold medals of any country (or ties for first) in the Paris Olympics.,Sports & Culture | The US wins the gold medal in basketball at the Paris Olympics.,"Sports & Culture | One of the three championship favorites (Boston Celtics, Denver Nuggets, Milwaukee Bucks) wins the 2024 NBA title.",Business & Tech | The S&P 500 closes the year higher than it began.,Business & Tech | The Fed raises interest rates at least once in 2024 (even if they also decrease rates).,Business & Tech | Sam Altman is still the CEO of OpenAI at the end of 2024.,"Business & Tech | OpenAI releases (or allows a public preview of) a product branded """"GPT-5"""" or """"ChatGPT-5"""".","Business & Tech | Meta is ordered to divest a business unit, by federal court or commission.","Business & Tech | Amazon is ordered to divest a business unit, by federal court or commission.","Business & Tech | Alphabet is ordered to divest a business unit, by federal court or commission.","Business & Tech | The X/Twitter platform still exists at the end of the year: you can log in, share content, and view content.",Business & Tech | Nvidia remains in the top 10 American companies by market cap at the end of 2024.,Business & Tech | Tim Cook is still the CEO of Apple at the end of the year.,Business & Tech | Andy Jassy is still the CEO of Amazon at the end of the year.
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2023/12/12 9:19:33 AM CST""","""Raluca Pavel""","""Yes""",0.26,0.15,0.0,1.0,1.0,0.9,1.0,0.8,1.0,0

In [4]:
raw = raw.drop(['Timestamp', "Can I track your answers on my website alongside others'?"])
raw = raw.rename({"What's your name?": "Name"})

In [5]:
no_resolutions = [
    'US Politics | Joe Biden is re-elected president.',
    "US Politics | A Republican candidate who isn't Donald Trump is elected president."
]
presumed_no_resolutions = [
    'Geopolitics | China lands ground forces on the island of Taiwan',
    'Geopolitics | Russia and Ukraine sign a truce to stop all combat operations and honor it until the end of the year',
    'Geopolitics | Nuclear weapons are used in an attack (not just tested) somewhere in the world.',
    "US Politics | A Democrat candidate who isn't Joe Biden is elected president.",
    'US Politics | 2024 elections give Democrats control of the House of Representatives.',
    'US Politics | At least one third-party US presidential candidate gets 3% or more of the popular vote.',
    'US Politics | The US president elect dies between the election and the end of 2024.',
    'US Politics | 2024 elections give Democrats control of the Senate.',
    'US Politics | At least one member of the US Supreme Court steps down or dies during 2024.',
    'US Politics | Donald Trump is convicted of a federal crime.',
    'US Politics | Joe Biden is impeached.',
    'Sports & Culture | Mainstream news outlets report definitively that Taylor Swift and Travis Kelce have broken up.',
    'Sports & Culture | LeBron James retires from or otherwise leaves the NBA, and does not change his mind before the end of the year.',
    'Sports & Culture | Lionel Messi retires from or otherwise leaves the MLS, and does not change his mind before the end of the year.',
    'Business & Tech | The Fed raises interest rates at least once in 2024 (even if they also decrease rates).',
    'Business & Tech | OpenAI releases (or allows a public preview of) a product branded ""GPT-5"" or ""ChatGPT-5"".',
    'Business & Tech | Meta is ordered to divest a business unit, by federal court or commission.',
    'Business & Tech | Amazon is ordered to divest a business unit, by federal court or commission.',
    'Business & Tech | Alphabet is ordered to divest a business unit, by federal court or commission.',
]

yes_resolutions = [
    'Sports & Culture | Taylor Swift releases an album of composed primarily of entirely new music.',
    'Sports & Culture | LeBron James and his son LeBron Jr (aka ""Bronny"") are under contract with the same NBA team at any point during 2024.',
    'Sports & Culture | The US wins the most gold medals of any country (or ties for first) in the Paris Olympics.',
    # thank goodness this is true whether it refers to men's or women's basketball
    'Sports & Culture | The US wins the gold medal in basketball at the Paris Olympics.',
    'Sports & Culture | One of the three championship favorites (Boston Celtics, Denver Nuggets, Milwaukee Bucks) wins the 2024 NBA title.',
]
presumed_yes_resolutions = [
    'Geopolitics | Xi Jinping is still the premier of China at the end of 2024.',
    'Geopolitics | Narendra Modi is still the prime minister of India at the end of 2024.',
    'Geopolitics | Benjamin Netanyahu is still the prime minister of Israel at the end of 2024.',
    'Geopolitics | Vladimir Putin is still the president of Russia at the end of 2024.',
    'Geopolitics | Vlodymyr Zelenskyy is still the president of Ukraine at the end of 2024.',
    'Geopolitics | Pope Francis (Jorge Mario Bergoglio) is still the active pope at the end of 2024.',
    'US Politics | Donald Trump is elected president.',
    'US Politics | President Jimmy Carter is still alive at the end of 2024.',
    'Business & Tech | The S&P 500 closes the year higher than it began.',
    'Business & Tech | Sam Altman is still the CEO of OpenAI at the end of 2024.',
    'Business & Tech | The X/Twitter platform still exists at the end of the year: you can log in, share content, and view content.',
    'Business & Tech | Nvidia remains in the top 10 American companies by market cap at the end of 2024.',
    'Business & Tech | Tim Cook is still the CEO of Apple at the end of the year.',
    'Business & Tech | Andy Jassy is still the CEO of Amazon at the end of the year.',   
]

# Temporarily, we have "presumptions" -- we'll get rid of them by the end of the year.
# In order to do calculations in the meantime, we'll just take our presumptions as accurate.
yes_resolutions = yes_resolutions + presumed_yes_resolutions
no_resolutions = no_resolutions + presumed_no_resolutions

In [6]:
# Make sure we included every question.
assert set(raw.columns) == set(yes_resolutions + no_resolutions + ['Name'])

In [7]:
props = [col for col in raw.columns if col != 'Name']
resolutions = pl.DataFrame({prop: float(prop in yes_resolutions) for prop in props})
resolutions

Geopolitics | China lands ground forces on the island of Taiwan,Geopolitics | Russia and Ukraine sign a truce to stop all combat operations and honor it until the end of the year,Geopolitics | Nuclear weapons are used in an attack (not just tested) somewhere in the world.,Geopolitics | Xi Jinping is still the premier of China at the end of 2024.,Geopolitics | Narendra Modi is still the prime minister of India at the end of 2024.,Geopolitics | Benjamin Netanyahu is still the prime minister of Israel at the end of 2024.,Geopolitics | Vladimir Putin is still the president of Russia at the end of 2024.,Geopolitics | Vlodymyr Zelenskyy is still the president of Ukraine at the end of 2024.,Geopolitics | Pope Francis (Jorge Mario Bergoglio) is still the active pope at the end of 2024.,US Politics | Donald Trump is elected president.,US Politics | Joe Biden is re-elected president.,US Politics | A Republican candidate who isn't Donald Trump is elected president.,US Politics | A Democrat candidate who isn't Joe Biden is elected president.,US Politics | At least one third-party US presidential candidate gets 3% or more of the popular vote.,US Politics | The US president elect dies between the election and the end of 2024.,US Politics | 2024 elections give Democrats control of the Senate.,US Politics | 2024 elections give Democrats control of the House of Representatives.,US Politics | At least one member of the US Supreme Court steps down or dies during 2024.,US Politics | President Jimmy Carter is still alive at the end of 2024.,US Politics | Donald Trump is convicted of a federal crime.,US Politics | Joe Biden is impeached.,Sports & Culture | Mainstream news outlets report definitively that Taylor Swift and Travis Kelce have broken up.,Sports & Culture | Taylor Swift releases an album of composed primarily of entirely new music.,"Sports & Culture | LeBron James retires from or otherwise leaves the NBA, and does not change his mind before the end of the year.","Sports & Culture | LeBron James and his son LeBron Jr (aka """"Bronny"""") are under contract with the same NBA team at any point during 2024.","Sports & Culture | Lionel Messi retires from or otherwise leaves the MLS, and does not change his mind before the end of the year.",Sports & Culture | The US wins the most gold medals of any country (or ties for first) in the Paris Olympics.,Sports & Culture | The US wins the gold medal in basketball at the Paris Olympics.,"Sports & Culture | One of the three championship favorites (Boston Celtics, Denver Nuggets, Milwaukee Bucks) wins the 2024 NBA title.",Business & Tech | The S&P 500 closes the year higher than it began.,Business & Tech | The Fed raises interest rates at least once in 2024 (even if they also decrease rates).,Business & Tech | Sam Altman is still the CEO of OpenAI at the end of 2024.,"Business & Tech | OpenAI releases (or allows a public preview of) a product branded """"GPT-5"""" or """"ChatGPT-5"""".","Business & Tech | Meta is ordered to divest a business unit, by federal court or commission.","Business & Tech | Amazon is ordered to divest a business unit, by federal court or commission.","Business & Tech | Alphabet is ordered to divest a business unit, by federal court or commission.","Business & Tech | The X/Twitter platform still exists at the end of the year: you can log in, share content, and view content.",Business & Tech | Nvidia remains in the top 10 American companies by market cap at the end of 2024.,Business & Tech | Tim Cook is still the CEO of Apple at the end of the year.,Business & Tech | Andy Jassy is still the CEO of Amazon at the end of the year.
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [8]:
averages = raw.mean()
averages = averages.with_columns(pl.lit('Average').alias("Name"))

estimates = raw.vstack(averages)
estimates.tail()

Name,Geopolitics | China lands ground forces on the island of Taiwan,Geopolitics | Russia and Ukraine sign a truce to stop all combat operations and honor it until the end of the year,Geopolitics | Nuclear weapons are used in an attack (not just tested) somewhere in the world.,Geopolitics | Xi Jinping is still the premier of China at the end of 2024.,Geopolitics | Narendra Modi is still the prime minister of India at the end of 2024.,Geopolitics | Benjamin Netanyahu is still the prime minister of Israel at the end of 2024.,Geopolitics | Vladimir Putin is still the president of Russia at the end of 2024.,Geopolitics | Vlodymyr Zelenskyy is still the president of Ukraine at the end of 2024.,Geopolitics | Pope Francis (Jorge Mario Bergoglio) is still the active pope at the end of 2024.,US Politics | Donald Trump is elected president.,US Politics | Joe Biden is re-elected president.,US Politics | A Republican candidate who isn't Donald Trump is elected president.,US Politics | A Democrat candidate who isn't Joe Biden is elected president.,US Politics | At least one third-party US presidential candidate gets 3% or more of the popular vote.,US Politics | The US president elect dies between the election and the end of 2024.,US Politics | 2024 elections give Democrats control of the Senate.,US Politics | 2024 elections give Democrats control of the House of Representatives.,US Politics | At least one member of the US Supreme Court steps down or dies during 2024.,US Politics | President Jimmy Carter is still alive at the end of 2024.,US Politics | Donald Trump is convicted of a federal crime.,US Politics | Joe Biden is impeached.,Sports & Culture | Mainstream news outlets report definitively that Taylor Swift and Travis Kelce have broken up.,Sports & Culture | Taylor Swift releases an album of composed primarily of entirely new music.,"Sports & Culture | LeBron James retires from or otherwise leaves the NBA, and does not change his mind before the end of the year.","Sports & Culture | LeBron James and his son LeBron Jr (aka """"Bronny"""") are under contract with the same NBA team at any point during 2024.","Sports & Culture | Lionel Messi retires from or otherwise leaves the MLS, and does not change his mind before the end of the year.",Sports & Culture | The US wins the most gold medals of any country (or ties for first) in the Paris Olympics.,Sports & Culture | The US wins the gold medal in basketball at the Paris Olympics.,"Sports & Culture | One of the three championship favorites (Boston Celtics, Denver Nuggets, Milwaukee Bucks) wins the 2024 NBA title.",Business & Tech | The S&P 500 closes the year higher than it began.,Business & Tech | The Fed raises interest rates at least once in 2024 (even if they also decrease rates).,Business & Tech | Sam Altman is still the CEO of OpenAI at the end of 2024.,"Business & Tech | OpenAI releases (or allows a public preview of) a product branded """"GPT-5"""" or """"ChatGPT-5"""".","Business & Tech | Meta is ordered to divest a business unit, by federal court or commission.","Business & Tech | Amazon is ordered to divest a business unit, by federal court or commission.","Business & Tech | Alphabet is ordered to divest a business unit, by federal court or commission.","Business & Tech | The X/Twitter platform still exists at the end of the year: you can log in, share content, and view content.",Business & Tech | Nvidia remains in the top 10 American companies by market cap at the end of 2024.,Business & Tech | Tim Cook is still the CEO of Apple at the end of the year.,Business & Tech | Andy Jassy is still the CEO of Amazon at the end of the year.
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""David Schmitz""",0.03,0.2,0.001,0.995,0.9,0.9,0.97,0.95,0.94,0.4,0.3,0.1,0.2,0.07,0.06,0.6,0.3,0.1,0.02,0.5,0.03,0.4,0.7,0.6,0.09,0.2,0.4,0.8,0.5,0.6,0.5,0.7,0.7,0.05,0.06,0.08,0.96,0.4,

In [9]:
from dataclasses import dataclass
from typing import Self

@dataclass
class Event:
    id: str
    category: str
    proposition: str

    @classmethod
    def from_id(cls, id: str) -> Self:
        category, proposition = id.split(' | ')
        return cls(id=id, category=category, proposition=proposition)

In [10]:
event_ids = [col for col in estimates.columns if col != 'Name']
events = [Event.from_id(event_id) for event_id in event_ids]
categories = {event.category for event in events}
categories

{'Business & Tech', 'Geopolitics', 'Sports & Culture', 'US Politics'}

In [11]:
people = estimates['Name']
people

Name
str
"""Raluca Pavel"""
"""Ethan Swan"""
"""Tommy Callan"""
"""Greg Moore"""
"""Abhisek Gupta"""
…
"""David Schmitz"""
"""Phil Anderson"""
"""Brian Van Fossen"""


Create a fresh DataFrame showing the Brier score contributions of each prediction.

In [12]:
props = [event.id for event in events]

rows = []
for person in people:
    resolutions = pl.Series(float(prop in yes_resolutions) for prop in props)
    predictions = pl.Series(estimates.filter(pl.col('Name') == person)[props].row(0))
    errors = (resolutions - predictions) ** 2
    rows.append(errors)

# Turn this back into a DataFrame.
df = pl.DataFrame(rows).transpose()
df.columns = props

# Add the "score" column: the average squared error.
df = df.with_columns(pl.mean_horizontal(df).alias('Score'))

# Add Name back in and reorder columns so it's first.
df = df.with_columns(pl.lit(people).alias('Name'))
df = df.select(['Name'] + props + ['Score'])
df.write_csv('generated/overall_scores.csv')
df

Name,Geopolitics | China lands ground forces on the island of Taiwan,Geopolitics | Russia and Ukraine sign a truce to stop all combat operations and honor it until the end of the year,Geopolitics | Nuclear weapons are used in an attack (not just tested) somewhere in the world.,Geopolitics | Xi Jinping is still the premier of China at the end of 2024.,Geopolitics | Narendra Modi is still the prime minister of India at the end of 2024.,Geopolitics | Benjamin Netanyahu is still the prime minister of Israel at the end of 2024.,Geopolitics | Vladimir Putin is still the president of Russia at the end of 2024.,Geopolitics | Vlodymyr Zelenskyy is still the president of Ukraine at the end of 2024.,Geopolitics | Pope Francis (Jorge Mario Bergoglio) is still the active pope at the end of 2024.,US Politics | Donald Trump is elected president.,US Politics | Joe Biden is re-elected president.,US Politics | A Republican candidate who isn't Donald Trump is elected president.,US Politics | A Democrat candidate who isn't Joe Biden is elected president.,US Politics | At least one third-party US presidential candidate gets 3% or more of the popular vote.,US Politics | The US president elect dies between the election and the end of 2024.,US Politics | 2024 elections give Democrats control of the Senate.,US Politics | 2024 elections give Democrats control of the House of Representatives.,US Politics | At least one member of the US Supreme Court steps down or dies during 2024.,US Politics | President Jimmy Carter is still alive at the end of 2024.,US Politics | Donald Trump is convicted of a federal crime.,US Politics | Joe Biden is impeached.,Sports & Culture | Mainstream news outlets report definitively that Taylor Swift and Travis Kelce have broken up.,Sports & Culture | Taylor Swift releases an album of composed primarily of entirely new music.,"Sports & Culture | LeBron James retires from or otherwise leaves the NBA, and does not change his mind before the end of the year.","Sports & Culture | LeBron James and his son LeBron Jr (aka """"Bronny"""") are under contract with the same NBA team at any point during 2024.","Sports & Culture | Lionel Messi retires from or otherwise leaves the MLS, and does not change his mind before the end of the year.",Sports & Culture | The US wins the most gold medals of any country (or ties for first) in the Paris Olympics.,Sports & Culture | The US wins the gold medal in basketball at the Paris Olympics.,"Sports & Culture | One of the three championship favorites (Boston Celtics, Denver Nuggets, Milwaukee Bucks) wins the 2024 NBA title.",Business & Tech | The S&P 500 closes the year higher than it began.,Business & Tech | The Fed raises interest rates at least once in 2024 (even if they also decrease rates).,Business & Tech | Sam Altman is still the CEO of OpenAI at the end of 2024.,"Business & Tech | OpenAI releases (or allows a public preview of) a product branded """"GPT-5"""" or """"ChatGPT-5"""".","Business & Tech | Meta is ordered to divest a business unit, by federal court or commission.","Business & Tech | Amazon is ordered to divest a business unit, by federal court or commission.","Business & Tech | Alphabet is ordered to divest a business unit, by federal court or commission.","Business & Tech | The X/Twitter platform still exists at the end of the year: you can log in, share content, and view content.",Business & Tech | Nvidia remains in the top 10 American companies by market cap at the end of 2024.,Business & Tech | Tim Cook is still the CEO of Apple at the end of the year.,Business & Tech | Andy Jassy is still the CEO of Amazon at the end of the year.,Score
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Raluca Pavel""",0.0676,0.0225,0.0,0.0,0.0,0.01,0.0,0.04,0.0,0.2704,0.2704,0.0,0.0,1.0,0.04,0.09,0.25,0.4225,1.0,1.0,0.0,0.49,0.81,0.0,0.64,0.01,0.0169,0.0,0.0025,0.1024,1.0,0.0

Build similar DataFrames, one for each category.

In [13]:
cat_dfs = {}
for cat in categories:
    cat_props = [event.id for event in events if event.category == cat]
    cat_df = df.select(['Name'] + cat_props)
    cat_df = cat_df.with_columns(pl.mean_horizontal(cat_df).alias('Score'))
    cat_dfs[cat] = cat_df
    category_name_cleaned = cat.replace(' ', '_').lower()
    cat_df.write_csv(f'generated/{category_name_cleaned}_scores.csv')
cat_dfs['Geopolitics']

Name,Geopolitics | China lands ground forces on the island of Taiwan,Geopolitics | Russia and Ukraine sign a truce to stop all combat operations and honor it until the end of the year,Geopolitics | Nuclear weapons are used in an attack (not just tested) somewhere in the world.,Geopolitics | Xi Jinping is still the premier of China at the end of 2024.,Geopolitics | Narendra Modi is still the prime minister of India at the end of 2024.,Geopolitics | Benjamin Netanyahu is still the prime minister of Israel at the end of 2024.,Geopolitics | Vladimir Putin is still the president of Russia at the end of 2024.,Geopolitics | Vlodymyr Zelenskyy is still the president of Ukraine at the end of 2024.,Geopolitics | Pope Francis (Jorge Mario Bergoglio) is still the active pope at the end of 2024.,Score
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Raluca Pavel""",0.0676,0.0225,0.0,0.0,0.0,0.01,0.0,0.04,0.0,0.015567
"""Ethan Swan""",0.0009,0.4225,0.01,0.0004,0.01,0.1225,0.0225,0.04,0.01,0.070978
"""Tommy Callan""",0.0,0.01,0.0,0.0,0.0,0.64,0.0,0.0,0.0,0.072222
"""Greg Moore""",0.0001,0.0025,0.0,0.0025,0.0025,0.01,0.01,0.01,0.04,0.008622
"""Abhisek Gupta""",0.0001,0.25,0.000001,0.01,0.0625,0.81,0.01,0.0625,0.01,0.135011
…,…,…,…,…,…,…,…,…,…,…
"""David Schmitz""",0.0009,0.04,0.000001,0.000025,0.01,0.01,0.0009,0.0025,0.0036,0.007547
"""Phil Anderson""",0.0001,0.01,1.0000e-8,0.0001,0.0025,0.0001,0.0001,0.09,0.09,0.021433
"""Brian Van Fossen""",0.0025,0.04,0.0001,0.04,0.09,0.09,0.04,0.0025,0.04,0.038344


Compute Brier scores, both for each category and overall.

In [14]:
import altair as alt

alt.Chart(df).mark_bar().encode(
    x=alt.X('Name', sort='y'),
    y='Score',
    color=alt.condition(
        alt.datum.Name == 'Average',
        alt.value('orange'),
        alt.value('blue'),
    ),
).properties(title='Overall Score')

alt.Chart(...)

In [15]:
def alt_charts_in_rows(charts: list[alt.Chart], n_cols: int) -> alt.Chart:
    """Display Altair charts in rows of n_cols."""
    n_rows = (len(charts) + n_cols - 1) // n_cols
    return alt.vconcat(*[alt.hconcat(*charts[i*n_cols:(i+1)*n_cols]) for i in range(n_rows)])

In [16]:
cat_charts = []
for category in categories:
    data = cat_dfs[category]
    chart = alt.Chart(data).mark_bar().encode(
        x=alt.X('Name', sort='y'),
        y='Score',
        color=alt.condition(
            alt.datum.Name == 'Average',
            alt.value('orange'),
            alt.value('blue'),
        ),
    ).properties(title=category)
    cat_charts.append(chart)

alt_charts_in_rows(cat_charts, 2)

alt.VConcatChart(...)

Score contributions

In [17]:
charts = []
for person in estimates['Name']:
    person_scores = df.filter(pl.col('Name') == person).drop('Score').unpivot(
        pl.selectors.numeric(),
        variable_name='prop',
        value_name='value',
    )
    chart = alt.Chart(person_scores).mark_bar().encode(
        x=alt.X('value', scale=alt.Scale(domain=[0, 1])),
        y=alt.Y('prop', sort='-x'),
        tooltip=['prop', 'value'],
    ).properties(
        title=person,
    )
    charts.append(chart)
    
alt_charts_in_rows(charts, 3)

alt.VConcatChart(...)